In [1]:
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import re
import time
import pyautogui
import matplotlib

# import skimage
from skimage import io, filters, exposure, measure, transform
from scipy.signal import find_peaks, savgol_filter
from scipy.optimize import linear_sum_assignment

pd.set_option('mode.chained_assignment',None)

# %matplotlib widget 
# %matplotlib inline
%matplotlib qt
# matplotlib.use('Qt5Agg')
matplotlib.rcParams.update({'figure.autolayout': True})

SMALLER_SIZE = 8
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [2]:
SCALE_100X = 15.8 # pix/µm 

In [3]:
# A = np.random.random((1000,1000))
# fig, ax = plt.subplots(1,1,figsize = (12,8))
# ax.imshow(A)
# fig.show()
# mngr = plt.get_current_fig_manager()
# mngr.window.setGeometry(-1900,100,900, 800)
# a = pyautogui.confirm(text='Are the beads of interest\npresent on the first image?', title='Initialise tracker', buttons=['Yes', 'No'])
# a

## Classes & functions

In [4]:
# experimentalDataDir = "C://Users//josep//Desktop//ActinCortexAnalysis//ExperimentalData"
experimentalDataDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//ExperimentalData"

def getExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
            if '.1' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            try:
                if expConditionsDF[col].dtype == 'string':
                    listTextColumns.append(col)
            except:
                aaaa=0
                #Ok

        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        try:
            expConditionsDF['optical index correction'] = \
                      expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                    / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        except:
            print('optical index correction already in ' + str(expConditionsDF['optical index correction'].dtype) + ' type.')

        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        try:
            expConditionsDF['ramp field'] = \
            expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])
        except:
            aaaa=0
            #Ok

        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
#     reorgaList = np.array([i for i in range(len(expConditionsDF.columns))])
#     reorgaList[2] = reorgaList[-1]
#     reorgaList[3:] = reorgaList[3:] - np.ones(len(reorgaList)-3)
#     expConditionsDF = expConditionsDF[expConditionsDF.columns[reorgaList]]
    
    return(expConditionsDF)

expDf = getExperimentalConditions()

expDf

Extracted a table with 40 lines and 24 columns.
optical index correction already in Float64 type.
dates corrected


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,loop structure,bacteria,comments,manipID
0,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,100X,15.8,oil,0.875,1.15,...,0,5,3_40,1s,False,3,<NA>,<NA>,<NA>,DEFAULT_DEFAULT
1,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-04_M1
2,20-08-04,M2,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-04_M2
3,20-08-05,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-05_M1
4,20-08-05,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-05_M2
5,20-08-07,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-07_M1
6,20-08-07,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-07_M2
7,21-01-18,M1,compressions,doxycyclin,20um fibronectin discs,100X,15.8,oil,0.875,1.23,...,0,5,3_40,1s,True,3,<NA>,<NA>,<NA>,21-01-18_M1
8,21-01-18,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.23,...,0,5,3_40,1s,True,3,<NA>,<NA>,<NA>,21-01-18_M2
9,21-01-18,M3,compressions,doxycyclin,20um fibronectin discs,100X,15.8,oil,0.875,1.23,...,0,5,3_40,1s,True,3,<NA>,<NA>,<NA>,21-01-18_M3


In [5]:
# a = pyautogui.confirm(text='Are the beads of interest\npresent on the first image?', title='Initialise tracker', buttons=['Yes', 'No'])
# a

In [6]:
def findInfosInFileName(f, infoType):
    if infoType in ['M', 'P', 'C']:
        acceptedChar = [str(i) for i in range(10)] + ['.', '-']
        string = '_' + infoType
        iStart = re.search(string, f).end()
        i = iStart
        infoString = '' + f[i]
        while f[i+1] in acceptedChar and i < len(f)-1:
            i += 1
            infoString += f[i]
    
    elif infoType == 'manipID':
        datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", f)
        date = f[datePos.start():datePos.end()]
        manip = 'M' + findInfosInFileName(f, 'M')
        infoString = date + '_' + manip
        
    elif infoType == 'cellID':
        datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", f)
        date = f[datePos.start():datePos.end()]
        infoString = date + '_' + 'M' + findInfosInFileName(f, 'M') + \
                            '_' + 'P' + findInfosInFileName(f, 'P') + \
                            '_' + 'C' + findInfosInFileName(f, 'C')
    
    return(infoString)

def isFileOfInterest(f, manips, wells, cells):
    test = False
    if f.endswith(".tif"):
        if manips == 'all':
            test = True
        else:
            try:
                manips_str = [str(i) for i in manips]
            except:
                manips_str = [str(manips)]
            infoM = findInfosInFileName(f, 'M')
            if infoM in manips_str:
                if wells == 'all':
                    test = True
                else:
                    try:
                        wells_str = [str(i) for i in wells]
                    except:
                        wells_str = [str(wells)]
                    infoP = findInfosInFileName(f, 'P')
                    if infoP in wells_str:
                        if cells == 'all':
                            test = True
                        else:
                            try:
                                cells_str = [str(i) for i in cells]
                            except:
                                cells_str = [str(cells)]
                            infoC = findInfosInFileName(f, 'C')
                            if infoC in cells_str:
                                test = True
    return(test)

def compute_cost_matrix(XY1,XY2):
    N1, N2 = XY1.shape[0],XY2.shape[0]
    M = np.zeros((N1, N2))
    for i in range(N1):
        for j in range(N2):
            M[i,j] = (np.sum((XY2[j,:] - XY1[i,:]) ** 2))
    return(M)

def ui2array(uixy):
    n = len(uixy)
    A = np.zeros((n, 2))
    for i in range(n):
        A[i,0], A[i,1] = uixy[i][0], uixy[i][1]
    return(A)
    

In [7]:
# a = 'aa12-32-12AZF34_M2'
# datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", a)
# a[datePos.start():datePos.end()]
# findInfosInFileName(a, 'manipID')
# a = 'aaa_bbb'
# a.split('_')
# s = '21-04-27_M1_P1_C4_R40_disc20um_wFluo'
# findInfosInFileName(s, 'cellID')
# [(331.26256664936864, 258.949997248057), (403.98986101188945, 258.9821671859072)]

XY1 = np.array([[1,2],[4,2],[7,2]])
XY2 = np.array([[4.2,1.9],[7.3,2.4],[10.5,2.9]])
M = compute_cost_matrix(XY1,XY2)
row_ind, col_ind = linear_sum_assignment(M)
for i in range(len(row_ind)):
    e1 = XY1[row_ind[i]]
    e2 = XY2[col_ind[i]]
    print(str(e1) + ' match with ' + str(e2))

[1 2] match with [4.2 1.9]
[4 2] match with [7.3 2.4]
[7 2] match with [10.5  2.9]


In [8]:
df = pd.DataFrame({'Area' : [1], 'StdDev' : [1], 'XM' : [1], 'YM' : [1], 'Slice' : [1]})
df.shape

(1, 5)

In [40]:
class PincherTimeLapse:
    
    def __init__(self, I, cellID, manipDict, NB = 2, ):
        nS, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        self.I = I
        self.threshold = 0
        self.NB = NB
        self.nx = nx
        self.ny = ny
        self.nS = nS
        self.listFrames = []
        self.listTrajectories = []
        self.dictLog = {'Slice' : np.array([i+1 for i in range(nS)]), \
                        'Status' : np.zeros(nS, dtype = int), \
                        # in the status field: -1 means excluded ; 0 means ramp ; >0 means position in the n-uplet
                        'Fluo' : np.zeros(nS, dtype = bool), \
                        'Black' : np.zeros(nS, dtype = bool), \
                        'UI' : np.zeros(nS, dtype = bool), \
                        'UILog' : np.array(['' for i in range(nS)],dtype = str), \
                        'UIxy' : np.zeros((nS,NB,2), dtype = int)}
        
        self.detectBeadsResult = pd.DataFrame(
            {'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})
        
        self.cellID = cellID
        self.expType = manipDict['experimentType']
        self.wFluo = bool(manipDict['with fluo images'])
        
        loopStruct = manipDict['loop structure'].split('_')
        self.loop_totalSize = int(loopStruct[0])
        if self.expType == 'compressions':
            self.loop_rampSize = int(loopStruct[1])
        else:
            self.loop_rampSize = 0
        if len(loopStruct) == 3: # This 3rd part of the 'loopStruct' field is the nb of frames at the end
        # of each loop which are not part of the main analysis and should be excluded. Typically fluo images.
            self.loop_excludedSize = int(loopStruct[2])
        else:
            self.loop_excludedSize = 0
        self.nLoop = int(np.round(nS/self.loop_totalSize))
        self.blackFramesPerLoop = np.zeros(self.nLoop)
        self.Nuplet = manipDict['normal field multi images']
        
    def checkIfBlackFrames(self):
        for i in range(self.nLoop):
            j = ((i+1)*self.loop_totalSize) - 1
            checkSum = np.sum(self.I[j])
            while checkSum == 0:
                self.dictLog['Black'][j] = True
                self.dictLog['Status'][j] = -1
                self.blackFramesPerLoop[i] += 1
                j -= 1
                checkSum = np.sum(self.I[j])
              
    def saveFluoAside(self, fluoDirPath = ''):
        if self.wFluo:
#             if not os.path.exists(fluoDirPath):
#                 os.makedirs(fluoDirPath)
            for i in range(self.nLoop):
                j = int(((i+1)*self.loop_totalSize) - 1 - self.blackFramesPerLoop[i])
                self.dictLog['Fluo'][j] = True
                self.dictLog['Status'][j] = -1
                
    def determineFramesStatus(self):
        N0 = self.loop_totalSize
        Nramp0 = self.loop_rampSize
        Nexclu = self.loop_excludedSize
        nUp = self.Nuplet
        N = N0 - Nexclu
        Nct = N - Nramp0
        for i in range(self.nLoop):
            jstart = int(i*N0)
            if Nramp0 == 0:
                for j in range(N):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
            else:
                Nramp = Nramp0-self.blackFramesPerLoop[i]
                for j in range(Nct//2):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                jstart += int(Nct//2 + Nramp)
                for j in range(Nct//2):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                
    def saveLog(self, display = 1, save = False, path = ''):
        dL = {}
        dL['Slice'], dL['Status'], dL['Fluo'], dL['Black'] = \
            self.dictLog['Slice'], self.dictLog['Status'], self.dictLog['Fluo'], self.dictLog['Black']
        dL['UI'], dL['UILog'] = \
            self.dictLog['UI'], self.dictLog['UILog']
        for i in range(self.NB):
            dL['UIx'+str(i+1)] = self.dictLog['UIxy'][:,i,0]
            dL['UIy'+str(i+1)] = self.dictLog['UIxy'][:,i,1]
        dfLog = pd.DataFrame(dL)
        if display == 1:
            print('\n\n* Initialized Log Table:\n')
            print(dfLog)
        if display == 2:
            print('\n\n* Filled Log Table:\n')
            print(dfLog[dfLog['UI']])
        if save:
            dfLog.to_csv(path, sep='\t')
        
    def importLog(self, path):
        dfLog = pd.read_csv(path, sep='\t')
        dL = dfLog.to_dict()
        self.dictLog['Slice'], self.dictLog['Status'], self.dictLog['Fluo'], self.dictLog['Black'] = \
            dfLog['Slice'].values, dfLog['Status'].values, dfLog['Fluo'].values, dfLog['Black'].values
        self.dictLog['UI'], self.dictLog['UILog'] = \
            dfLog['UI'].values, dfLog['UILog'].values
        for i in range(self.NB):
            xkey, ykey = 'UIx'+str(i+1), 'UIy'+str(i+1)
            self.dictLog['UIxy'][:,i,0] = dfLog[xkey].values
            self.dictLog['UIxy'][:,i,1] = dfLog[ykey].values
        
    def computeThreshold(self, method = 'otsu'):
        # TBC
#         factorT = 0.8*(self.D == 4.5) + 0.6*(self.D == 2.7)
        factorT = 0.35
        threshold = factorT*filters.threshold_otsu(self.I)
        self.threshold = threshold
        
    def testThresholding(self):
        I_test = self.I[self.nS//2]
        I_thresh = I_test > self.threshold
        fig, ax = plt.subplots(1,1)
        ax.imshow(I_thresh, cmap = 'gray')
        fig.show()
        
    def makeFramesList(self):
        for i in range(self.nS):
            status = self.dictLog['Status'][i]
            if self.dictLog['Status'][i] >= 0:
                self.listFrames.append(Frame(self.I[i], i, self.NB, self.threshold, self.Nuplet, status))
    
    def detectBeads(self, resFileImported, display = False):
        for frame in self.listFrames: #[:3]:
            if not resFileImported:
                frame.detectBeads()
                self.detectBeadsResult = pd.concat([self.detectBeadsResult, frame.resDf])
            else:
                resDf = self.detectBeadsResult.loc[self.detectBeadsResult['Slice'] == frame.iS+1]
                frame.resDf = resDf
                
            frame.makeListBeads()
            
        if not resFileImported:
            self.detectBeadsResult = self.detectBeadsResult.convert_dtypes()
            self.detectBeadsResult.reset_index(inplace=True)
            self.detectBeadsResult.drop(['index'], axis = 1, inplace=True)
        
        if display:
            print('\n\n* Detected Beads Result:\n')
            print(self.detectBeadsResult)

        
#     def makeBeadsDetectResult(self, save=False, path=''):
#         df = pd.DataFrame({'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})
#         df = pd.concat([pd.DataFrame(frame.resDict) for frame in self.listFrames])
#         df = df.reset_index()
#         df = df.drop(['index'], axis = 1) # To be checked
#         if save:
#             df.to_csv(path, sep=';')
#         return(df)

    def saveBeadsDetectResult(self, path=''):
        self.detectBeadsResult.to_csv(path, sep='\t')
    
    def importBeadsDetectResult(self, path=''):
        df = pd.read_csv(path, sep='\t')
        for c in df.columns:
            if 'Unnamed' in c:
                df.drop([c], axis = 1, inplace=True)
        self.detectBeadsResult = df
        
    def findBestStd(self):
        """
        This ugly function is my best attempt to implement sth very simple in a robust way.
        In the 'status' field, -1 means excluded image, 0 means image that isn't part of a N-uplet of images, and k>0 means position in the N-uplet of images.
        For each image in the N-uplet, I want to reconsititute this N-uplet (meaning the list of Nuplet consecutive images numbered from 1 to Nuplet, minus the images eventually with no beads detected).
        Then for each N-uplet of images, i want to find the max standard deviation and report its position because it's for the max std that the X and Y detection is the most precise.
        An exemple: with these inputs:
        Nuplet = 3
        status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
            iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
           std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
        The function will return bestStd, a list of boolean with the same length.
        Where status = 0, bestStd = True (the image is not part of a N-uplet, thus it need to be analysed regardless of its std).
        Where satus > 0, the function will cut the lists in N-uplet of max size 3:
        status -> [1,2,3] ; [1,2] ; [ 3] ; [ 1, 2, 3] ; [ 1, 2]
            iS -> [0,1,2] ; [6,7] ; [11] ; [12,13,14] ; [15,16]
           std -> [1,5,9] ; [1,5] ; [ 8] ; [ 2, 6, 9] ; [ 2, 6]
             i -> [0,1,2] ; [6,7] ; [ 8] ; [ 9,10,11] ; [12,13]
        and then will find the best std in each of those fragment and put a True at that position (list 'i') in bestStd, so in this example, at i = 2, 7, 8, 11 ,13
        So the output is: bestStd = [False, False, True, True, True, True, False, True, True, False, False, True, False, True]
        """
        
        Nuplet = self.Nuplet
        status = self.listTrajectories[0].dict['Status']
        iS = self.listTrajectories[0].dict['iS']
        Ntraj = len(iS)
        std = np.zeros(Ntraj)
        for i in range(0,self.NB):
            std += np.array(self.listTrajectories[i].dict['StdDev'])
        
        bestStd = np.zeros(Ntraj, dtype = bool)

        current_Nup_status = []
        current_Nup_iS = []
        current_Nup_std = []
        current_Nup_i = []
        for i in range(Ntraj):
            if status[i] == 0:
                bestStd[i] = True
            else:
                if len(current_Nup_i) == 0:
                    current_Nup_status = [status[i]]
                    current_Nup_iS = [iS[i]]
                    current_Nup_std = [std[i]]
                    current_Nup_i = [i]
                else:
                    if status[i] > current_Nup_status[-1] and (iS[i]-current_Nup_iS[-1]) < Nuplet:
                        current_Nup_status.append(status[i])
                        current_Nup_iS.append(iS[i])
                        current_Nup_std.append(std[i])
                        current_Nup_i.append(i)
                    else:
    #                     print(current_Nup_status, current_Nup_iS, current_Nup_std, current_Nup_i)
                        i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
    #                     print(i_bestStdInCurrentNuplet)
                        i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
                        bestStd[i_bestStd] = True
                        # then:
                        current_Nup_status = [status[i]]
                        current_Nup_iS = [iS[i]]
                        current_Nup_std = [std[i]]
                        current_Nup_i = [i]

        # Need to do it one last time at the end
        i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
        i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
        bestStd[i_bestStd] = True

        return(bestStd)
        
    def buildTrajectories(self):
        # NB: 'iF': index in a list of Frames ; 'iB': index in list of Beads or Trajectories ; 'iS': index of the slice in the image I
        # Boi refers to the 'Beads of interest', ie the beads that are being tracked
        init_iF = 0
        init_ok = False
        while not init_ok:
            init_iS = self.listFrames[init_iF].iS
            if not self.dictLog['UI'][init_iS]:
                self.listFrames[init_iF].show()
                mngr = plt.get_current_fig_manager()
                mngr.window.setGeometry(700,100,1000, 800)
                QA = pyautogui.confirm(
                    text='Can you point the beads of interest\nin the image ' + str(init_iS + 1) + '?',
                    title='Initialise tracker', 
                    buttons=['Yes', 'Next Frame', 'Quit'])
                if QA == 'Yes':
                    init_ok = True
                    ui = plt.ginput(2, timeout=0)
                    uiXY = ui2array(ui)
                    fig = plt.gcf()
                    plt.close(fig)
                    self.dictLog['UI'][init_iS] = True
                    self.dictLog['UILog'][init_iS] = 'init_' + QA
                    self.dictLog['UIxy'][init_iS] = uiXY
                elif QA == 'Next Frame':
                    fig = plt.gcf()
                    plt.close(fig)
                    self.dictLog['UI'][init_iS] = True
                    self.dictLog['UILog'][init_iS] = 'init_' + QA
                    init_iF += 1
                else:
                    fig = plt.gcf()
                    plt.close(fig)
                    return('Bug')
            else:
                QA = self.dictLog['UILog'][init_iS]
                if QA == 'init_Yes':
                    init_ok = True
                    uiXY = self.dictLog['UIxy'][init_iS]
                elif QA == 'Next Frame':
                    init_iF += 1
                else:
                    aa
        
        init_BXY = self.listFrames[init_iF].beadsXYarray()
        M = compute_cost_matrix(uiXY,init_BXY)
        row_ind, col_ind = linear_sum_assignment(M) # row_ind -> clicks / col_ind -> listBeads
        # Sort the initial beads to have them ordered by growing x coord.
        sortM = np.array([[init_BXY[col_ind[i],0], col_ind[i]] for i in range(len(col_ind))])
        sortM = sortM[sortM[:, 0].argsort()]
        init_iBoi = sortM[:, 1].astype(int)
        init_BoiXY = np.array([init_BXY[col_ind[i]] for i in range(len(col_ind))])
        
        for iB in range(self.NB):
            self.listTrajectories.append(Trajectory(self.I))
#             self.listTrajectories[iB].seriesBeads.append(self.listFrames[init_iF].listBeads[col_ind[iB]])
#             self.listTrajectories[iB].pointerBeads.append(init_iBoi[iB])
#             self.listTrajectories[iB].series_iS.append(self.listFrames[init_iF].iS)
#             self.listTrajectories[iB].seriesXY.append(init_BoiXY[iB].tolist())
#             self.listTrajectories[iB].current_iS = self.listFrames[init_iF].iS
            #
            self.listTrajectories[iB].dict['Bead'].append(self.listFrames[init_iF].listBeads[init_iBoi[iB]])
            self.listTrajectories[iB].dict['iF'].append(init_iF)
            self.listTrajectories[iB].dict['iS'].append(self.listFrames[init_iF].iS)
            self.listTrajectories[iB].dict['iB'].append(init_iBoi[iB])
            self.listTrajectories[iB].dict['X'].append(init_BoiXY[iB][0])
            self.listTrajectories[iB].dict['Y'].append(init_BoiXY[iB][1])
            self.listTrajectories[iB].dict['StdDev'].append(self.listFrames[init_iF].beadsStdDevarray()[init_iBoi[iB]])
            self.listTrajectories[iB].dict['Status'].append(self.listFrames[init_iF].status)
            self.listTrajectories[iB].dict['idxRamp'].append(1 * (self.listFrames[init_iF].status == 0))

        previous_iF = init_iF
        previous_iBoi = init_iBoi
        previous_BXY = init_BXY
        previous_BoiXY = init_BoiXY
        
        for iF in range(init_iF+1, len(self.listFrames)):
            validFrame = True
            
            if len(self.listFrames[iF].listBeads) < self.NB:
                validFrame = False
            
            else:
                BXY = self.listFrames[iF].beadsXYarray()
                M = compute_cost_matrix(previous_BXY,BXY)
                row_ind, col_ind = linear_sum_assignment(M)
                costs = [M[row_ind[iB],col_ind[iB]] for iB in range(len(row_ind))]
                
                # Assess wether the algo should aks for user input
                askUI = False
                if (max(costs)**0.5) * (1/SCALE_100X) > 1: # If the greatest distance travelled by any object is greater than 1um
                    askUI = True

                if not askUI: # Automatically asign the positions of the next beads
                    iBoi = [col_ind[row_ind.tolist().index(iB)] for iB in previous_iBoi]
                    BoiXY = np.array([BXY[iB] for iB in iBoi])

                elif askUI: # Ask user input to asign the positions of the next beads
                    iS = self.listFrames[iF].iS
                    # Case 1: the UI has not been previously saved in the dictLog
                    # Then ask for UI and save it in the dictLog
                    if not self.dictLog['UI'][iS]:
                        # Display the image, plot beads positions and current trajectories & ask the question
                        self.listFrames[iF].show()
                        for iB in range(self.NB):
                            T = self.listTrajectories[iB]
                            ax = plt.gca()
                            T.plot(ax, iB)
                        mngr = plt.get_current_fig_manager()
                        mngr.window.setGeometry(700,100,1000, 800)
                        QA = pyautogui.confirm(
                            text='Can you point the beads of interest\nin the image ' + str(iS + 1) + '?',
                            title='', 
                            buttons=['No', 'Yes', 'Abort Mission!'])
                        
                        # According to the question's answer:
                        if QA == 'Yes':
                            ui = plt.ginput(2, timeout=0)
                            uiXY = ui2array(ui)
                            fig = plt.gcf()
                            plt.close(fig)
                            self.dictLog['UI'][iS] = True
                            self.dictLog['UILog'][iS] = QA
                            self.dictLog['UIxy'][iS] = uiXY
                        elif QA == 'No':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
                            self.dictLog['UI'][iS] = True
                            self.dictLog['UILog'][iS] = QA
                        elif QA == 'Abort Mission!':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
                            return('Bug')
                    
                    # Case 2: the UI has been previously saved in the dictLog.
                    # Then just import the previous answer from the dictLog
                    else:
                        QA = self.dictLog['UILog'][iS]
                        if QA == 'Yes':
                            uiXY = self.dictLog['UIxy'][iS]
                            
                        elif QA == 'No':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
            
            # If there were more than NB objects, and then the QA wasn't 'No', the frame is valid.
            if validFrame:
                M = compute_cost_matrix(uiXY,BXY)
                row_ind, col_ind = linear_sum_assignment(M)
                sortM = np.array([[BXY[col_ind[i],0], col_ind[i]] for i in range(len(col_ind))])
                sortM = sortM[sortM[:, 0].argsort()]
                iBoi = sortM[:, 1].astype(int)
                BoiXY = np.array([BXY[iB] for iB in iBoi])
                # idxRamp = 0 if not in a ramp, and = number of ramp else. Basically increase by 1 each time you have an interval between two ramps.
                idxRamp = (self.listFrames[iF].status == 0) * (max(self.listTrajectories[iB].dict['idxRamp']) + 1 * (self.listTrajectories[iB].dict['idxRamp'][-1] == 0))
                for iB in range(self.NB):
#                     self.listTrajectories[iB].seriesBeads.append(self.listFrames[iF].listBeads[iBoi[iB]])
#                     self.listTrajectories[iB].pointerBeads.append(iBoi[iB])
#                     self.listTrajectories[iB].series_iS.append(self.listFrames[iF].iS)
#                     self.listTrajectories[iB].seriesXY.append(BoiXY[iB].tolist())
#                     self.listTrajectories[iB].current_iS = self.listFrames[iF].iS
                    #
                    self.listTrajectories[iB].dict['Bead'].append(self.listFrames[iF].listBeads[iBoi[iB]])
                    self.listTrajectories[iB].dict['iF'].append(iF)
                    self.listTrajectories[iB].dict['iS'].append(self.listFrames[iF].iS)
                    self.listTrajectories[iB].dict['iB'].append(iBoi[iB])
                    self.listTrajectories[iB].dict['X'].append(BoiXY[iB][0])
                    self.listTrajectories[iB].dict['Y'].append(BoiXY[iB][1])
                    self.listTrajectories[iB].dict['StdDev'].append(self.listFrames[iF].beadsStdDevarray()[iBoi[iB]])
                    self.listTrajectories[iB].dict['Status'].append(self.listFrames[iF].status)
                    self.listTrajectories[iB].dict['idxRamp'].append(idxRamp)
                    
                previous_iF = iF
                previous_iBoi = iBoi
                previous_BXY = BXY
                previous_BoiXY = BoiXY
                
        for iB in range(self.NB):
            for k in self.listTrajectories[iB].dict.keys():
                self.listTrajectories[iB].dict[k] = np.array(self.listTrajectories[iB].dict[k])
                
        # Now we have a functional Trajectory object
        # Time to refine it
        
        Ntraj = len(self.listTrajectories[0].dict['Bead'])
        
        # (a) Add the pointer to the correct line of the _Field.txt file.
        # It's just exactly the iS already saved in the dict, except if there are black images at the end of loops.
        # In that case you have to skip the X lines corresponding to the end of the ramp part, X being the nb of black images at the end of the current loop
        # This is because when black images occurs, they do because of the high frame rate during ramp parts and thus replace these last ramp images.
        
        for iB in range(self.NB):
            iField = []
            for i in range(Ntraj):
                S = self.listTrajectories[iB].dict['iS'][i]
                iLoop = (S//self.loop_totalSize)
                offset = self.blackFramesPerLoop[iLoop]
                i_lim = iLoop*self.loop_totalSize + (self.loop_totalSize - ((self.loop_totalSize-self.loop_rampSize)//2) - (self.loop_excludedSize + offset))
                # i_lim is the first index after the end of the ramp
                addOffset = (S >= i_lim)
                SField = S + int(addOffset*offset)
                iField.append(SField)
            self.listTrajectories[iB].dict['iField'] = iField
            
        # (b) Find the image with the best std within each n-uplet
            
        bestStd = self.findBestStd()
        
        for i in range(self.NB):
            self.listTrajectories[i].dict['bestStd'] = bestStd

In [41]:
# Test findBestStd

# def findBestStd(Nuplet, status, iS, std):
#     """
#     This ugly function is my best attempt to implement sth very simple in a robust way.
#     In the 'status' field, -1 means excluded image, 0 means image that isn't part of a N-uplet of images, and k>0 means position in the N-uplet of images.
#     For each image in the N-uplet, I want to reconsititute this N-uplet (meaning the list of Nuplet consecutive images numbered from 1 to Nuplet, minus the images eventually with no beads detected).
#     Then for each N-uplet of images, i want to find the max standard deviation and report its position because it's for the max std that the X and Y detection is the most precise.
#     An exemple: with these inputs:
#     Nuplet = 3
#     status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
#         iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
#        std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
#     The function will return bestStd, a list of boolean with the same length.
#     Where status = 0, bestStd = True (the image is not part of a N-uplet, thus it need to be analysed regardless of its std).
#     Where satus > 0, the function will cut the lists in N-uplet of max size 3:
#     status -> [1,2,3] ; [1,2] ; [ 3] ; [ 1, 2, 3] ; [ 1, 2]
#         iS -> [0,1,2] ; [6,7] ; [11] ; [12,13,14] ; [15,16]
#        std -> [1,5,9] ; [1,5] ; [ 8] ; [ 2, 6, 9] ; [ 2, 6]
#          i -> [0,1,2] ; [6,7] ; [ 8] ; [ 9,10,11] ; [12,13]
#     and then will find the best std in each of those fragment and put a True at that position (list 'i') in bestStd, so in this example, at i = 2, 7, 8, 11 ,13
#     So the output is: bestStd = [False, False, True, True, True, True, False, True, True, False, False, True, False, True]
#     """
    
#     Ntraj = len(iS)
#     bestStd = np.zeros(Ntraj, dtype = bool)
#     M = np.array([status, iS, std])
#     current_Nup_status = []
#     current_Nup_iS = []
#     current_Nup_std = []
#     current_Nup_i = []
#     for i in range(Ntraj):
#         if status[i] == 0:
#             bestStd[i] = True
#         else:
#             if len(current_Nup_i) == 0:
#                 current_Nup_status = [status[i]]
#                 current_Nup_iS = [iS[i]]
#                 current_Nup_std = [std[i]]
#                 current_Nup_i = [i]
#             else:
#                 if status[i] > current_Nup_status[-1] and (iS[i]-current_Nup_iS[-1]) < Nuplet:
#                     current_Nup_status.append(status[i])
#                     current_Nup_iS.append(iS[i])
#                     current_Nup_std.append(std[i])
#                     current_Nup_i.append(i)
#                 else:
# #                     print(current_Nup_status, current_Nup_iS, current_Nup_std, current_Nup_i)
#                     i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
# #                     print(i_bestStdInCurrentNuplet)
#                     i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
#                     bestStd[i_bestStd] = True
#                     # then:
#                     current_Nup_status = [status[i]]
#                     current_Nup_iS = [iS[i]]
#                     current_Nup_std = [std[i]]
#                     current_Nup_i = [i]
                    
#     # Need to do it one last time at the end
#     i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
#     i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
#     bestStd[i_bestStd] = True
    
#     return(bestStd)

# Nuplet = 3
# status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
# iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
# std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
# bs = findBestStd(Nuplet, status, iS, std)
# bs

In [42]:
class Frame:
    
    def __init__(self, F, iS, NB, threshold, Nup, status):
        ny, nx = F.shape[0], F.shape[1]
        self.F = F # Note : Frame.F points directly to the i-th frame of the image I ! To have 2 different versions one should use np.copy(F)
        self.threshold = threshold
        self.NBoi = NB
        self.NBdetected = 0
        self.nx = nx
        self.ny = ny
        self.iS = iS
        self.listBeads = []
        self.trajPoint = []
        self.Nuplet = Nup
        self.status = status
        self.resDf = pd.DataFrame({'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})

        
    def __str__(self):
        text = 'a'
        return(text)
    
    def show(self, strech = True):
        fig, ax = plt.subplots(1,1)
#         fig_size = plt.gcf().get_size_inches()
#         fig.set_size_inches(2 * fig_size)
        if strech:
            pStart, pStop = np.percentile(self.F, (1, 99))
            ax.imshow(self.F, cmap = 'gray', vmin = pStart, vmax = pStop)
        else:
            ax.imshow(self.F, cmap = 'gray')
        if len(self.listBeads) > 0:
            for B in self.listBeads:
                ax.plot([B.x], [B.y], c='orange', marker='o')
        fig.show()
    
    def detectBeads(self):
        F_bin = self.F > self.threshold
        F_lab, nObj = ndi.label(F_bin)
        props = measure.regionprops(F_lab)
        listValidLabels = []
        areas = np.zeros(nObj+1)
        for k in range(1, nObj+1):
            try:
                bb = props[k-1].bbox
                Valid = not (min(bb) == 0 or bb[2] == self.ny or bb[3] == self.nx) # Remove objects touching the edges of the frame

    #             # OPTION 1 - Compute the metrics on the filled shape ; NB: takes a lot of time
    #             F_fh = ndi.binary_fill_holes((F_lab == k).astype(int))
    #             tmp_props = measure.regionprops(F_fh.astype(int))
    #             A = tmp_props[0].area
    #             P = tmp_props[0].perimeter
    #             Circ = (4 * np.pi * A) / (P * P)
    #             Valid = Valid and A >= 100 and Circ >= 0.75 # Area and circularity criterion

                # OPTION 2 - Lower the criterion in circularity - NB: less selective
                A = props[k-1].area
                P = props[k-1].perimeter
                Circ = (4 * np.pi * A) / (P * P)
                Valid = Valid and A >= 100 and Circ >= 0.65 # Area and circularity criterion

            except:
                Valid = False
            if Valid:
                listValidLabels.append(k)
                areas[k] = A
                
#         F_labValid, nObjValid = ndi.label(F_lab)    
#         fig, ax = plt.subplots(1,1)
#         ax.imshow(F_labValid)
#         fig.show()
        
        centoids = np.array([ndi.center_of_mass(self.F, labels=F_lab, index=i) for i in listValidLabels])
        resDict = {}
        resDict['Area'] = np.array([areas[i] for i in listValidLabels]).astype(int)
        resDict['StdDev'] = np.array([ndi.standard_deviation(self.F, labels=F_lab, index=i) for i in listValidLabels])
        resDict['XM'] = centoids[:,1]
        resDict['YM'] = centoids[:,0]
        resDict['Slice'] = np.array([self.iS+1 for i in listValidLabels]).astype(int)
        self.resDf = pd.DataFrame(resDict)
        
#         print(self.resDict)

    def makeListBeads(self):
        self.NBdetected = self.resDf.shape[0]
        for i in range(self.NBdetected):
            d = {}
            for c in self.resDf.columns:
                d[c] = self.resDf[c].values[i]
            self.listBeads.append(Bead(d, self.F))
            
    def beadsXYarray(self):
        A = np.zeros((len(self.listBeads), 2))
        for i in range(len(self.listBeads)):
            b = self.listBeads[i]
            A[i,0], A[i,1] = b.x, b.y
        return(A)
    
    def beadsStdDevarray(self):
        A = np.zeros(len(self.listBeads))
        for i in range(len(self.listBeads)):
            b = self.listBeads[i]
            A[i] = b.std
        return(A)

In [43]:
class Bead:
    
    def __init__(self, d, F):
        self.x = d['XM']
        self.y = d['YM']
        self.R = 0
        self.area = d['Area']
        self.std = d['StdDev']
        self.iS = d['Slice']-1
        self.status = ''
        self.hasRightNeighbour = False
        self.hasLeftNeighbour = False
        self.F = F

    def show(self, strech = True):
        fig, ax = plt.subplots(1,1)
        if strech:
            pStart, pStop = np.percentile(self.F, (1, 99))
            ax.imshow(self.F, cmap = 'gray', vmin = pStart, vmax = pStop)
        else:
            ax.imshow(self.F, cmap = 'gray')
        ax.plot([self.x], [self.y], c='orange', marker='o')
        fig.show()

In [44]:
class Trajectory:
    def __init__(self, I):
        nS, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        self.I = I
        self.nx = nx
        self.ny = ny
        self.nS = nS
        self.R = 0
#         self.seriesBeads = []
#         self.pointerBeads = []
#         self.series_iS = []
#         self.seriesXY = []
#         self.current_iS = 0
        self.dict = {'X': [],'Y': [],'idxRamp': [],'StdDev': [], \
                     'Bead': [],'Status': [],'iF': [],'iS': [],'iB' : [],}
        # These columns will be added later : 'Neighbours', 'Z', 'bestStd' and 'iField'
        
    def __str__(self):
        text = 'iS : ' + str(self.series_iS)
        text += '\n'
        text += 'XY : ' + str(self.seriesXY)
        return(text)
    
    def to_df(self):
        df = pd.DataFrame(self.dict)
        return(df)
    
    def plot(self, ax, i_color):
        colors = ['cyan', 'red', 'blue', 'orange']
        c = colors[i_color]
        ax.plot(self.dict['X'], self.dict['Y'], color=c, lw=0.5)

In [45]:
# f = Frame(np.array([[1, 2], [3, 4]]), 1, 2, 1000)
# print(f)
# A = np.array([[1, 2], [3, 4]])
# np.sum(A)

In [46]:
A = [1,1,1]
str(A)

'[1, 1, 1]'

In [47]:
mainDataDir = 'D://MagneticPincherData'
# mainDataDir = 'C://Users//josep//Desktop//TestData_BeadTracker'
rawDataDir = os.path.join(mainDataDir, 'Raw')
interDataDir = os.path.join(mainDataDir, 'Intermediate')
figureDir = os.path.join(mainDataDir, 'Figures')
dateTest1 = '21-04-27'
dateTest2 = '21.04.27'
# 21-04-27_M1_P1_C4_R40_disc20um_wFluo
# imagesToAnalyse.append(os.path.join(rd, f))

In [48]:
def main():
    
    start = time.time()
    
    dates = '21.01.21'
#     dates = '21.04.27'
#     manip = 'M1_P1_C4'
#     manipID = '21-04-27_M1'
    
    manips = 1
    wells = 1
    cells = 1
    
    
    
    # 0. Load different data sources & Preprocess : fluo, black images, sort slices (ct/ramp ; down/middle/up)
        # Make list of files to analyse
    imagesToAnalyse = []
    imagesToAnalyse_Paths = []
    if not isinstance(dates, str):
        rawDirList = [os.path.join(rawDataDir, d) for d in dates]
    else:
        rawDirList = [os.path.join(rawDataDir, dates)]
    for rd in rawDirList:
        fileList = os.listdir(rd)
        for f in fileList:
            if isFileOfInterest(f, manips, wells, cells):
                imagesToAnalyse.append(f)
                imagesToAnalyse_Paths.append(os.path.join(rd, f))          
    
    # Begining of the MAIN LOOP
    for i in range(len(imagesToAnalyse)): 
        f, fP = imagesToAnalyse[i], imagesToAnalyse_Paths[i]
        manipID = findInfosInFileName(f, 'manipID')
        cellID = findInfosInFileName(f, 'cellID')
        
        # Load exp data
        if manipID not in expDf['manipID'].values:
            print('Error! No experimental data found for: ' + manipID)
            bug
        else:
            expDf_line = expDf.loc[expDf['manipID'] == manipID]
            manipDict = {}
            for c in expDf_line.columns.values:
                manipDict[c] = expDf_line[c].values[0]
    
        # Load image and init PTL
        I = io.imread(fP) # Approx 0.5s per image
        PTL = PincherTimeLapse(I, cellID, manipDict, NB = 2)
    
        # Load field file
        fieldFilePath = fP[:-4] + '_Field.txt'
        fieldCols = ['B_set', 'T_abs', 'B', 'Z']
        fieldDf = pd.read_csv(fieldFilePath, sep = '\t', names = fieldCols)
        
        # Check if a log file exists and load it if required
        logFilePath = fP[:-4] + '_Log.txt'
        logFileImported = False
        if os.path.isfile(logFilePath):
            PTL.importLog(logFilePath)
            PTL.dictLog['UILog'] = PTL.dictLog['UILog'].astype(str)
            logFileImported = True
        
        # Detect fluo & black images
        PTL.checkIfBlackFrames()
        PTL.saveFluoAside()
        if not logFileImported:
            pass # I may change the organization of this part later
        
        # Sort slices
        if not logFileImported:
            PTL.determineFramesStatus()
        
        PTL.saveLog(display = True, save = (not logFileImported), path = logFilePath)
        
        # Determine global threshold
        PTL.computeThreshold() # Approx 3s per image
#         PTL.testThresholding()
        
        # Create list of Frame objects
        PTL.makeFramesList()
    
    
    # 1. Detect beads
        # Check if a _Results.txt exists and import it if so
        resFilePath = fP[:-4] + '_ResultsPY.txt'
        resFileImported = False
        if os.path.isfile(resFilePath):
            PTL.importBeadsDetectResult(resFilePath)
            resFileImported = True
    
        # Dectect the beads and create the BeadsDetectResult dataframe [if no file has been loaded before] 
        # OR input the results in each Frame objects [if the results have been loaded at the previous step]
        PTL.detectBeads(resFileImported, display = 1)
        
        # Save the new results if necessary
        if not resFileImported:
            PTL.saveBeadsDetectResult(path=resFilePath)

        
    # 2. Make trajectories for beads of interest
        # Display the first image to ask for the 2 beads of interest
        issue = PTL.buildTrajectories()
        if issue == 'Bug':
            continue
        else:
            pass
        PTL.saveLog(display = 2, save = True, path = logFilePath)
        
        # 
    

        # 
        
        
        # 
        
        
    # 3. Qualify - Detect boi sizes and neighbours
    # 4. Define pairs and compute dx, dy
    # 5. Compute dz
    # 6. Convert in real units
    # 7. Compute forces
    # 8. Export the results
    print('\nTotal time:')
    print(time.time()-start)
    return(PTL)

In [49]:
PTL = main()

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'




* Initialized Log Table:

      Slice  Status   Fluo  Black     UI     UILog  UIx1  UIy1  UIx2  UIy2
0         1       1  False  False   True  init_Yes   262   343   350   371
1         2       2  False  False  False       nan     0     0     0     0
2         3       3  False  False  False       nan     0     0     0     0
3         4       1  False  False  False       nan     0     0     0     0
4         5       2  False  False  False       nan     0     0     0     0
...     ...     ...    ...    ...    ...       ...   ...   ...   ...   ...
1355   1356       1  False  False  False       nan     0     0     0     0
1356   1357       2  False  False  False       nan     0     0     0     0
1357   1358       3  False  False  False       nan     0     0     0     0
1358   1359      -1   True  False  False       nan     0     0     0     0
1359   1360      -1  False   True  False       nan     0     0     0     0

[1360 rows x 10 columns]


* Detected Beads Result:

      Area        

In [ ]:
frame = PTL.listFrames[-5].show()
ax = plt.gca()
for i in range(PTL.NB):
    PTL.listTrajectories[i].plot(ax, i)
plt.show()

In [ ]:
traj = PTL.listTrajectories[1]
traj.to_df()

In [ ]:
traj = PTL.listTrajectories[1]
a = np.sum(np.array(traj.dict['idxRamp']) == 1)
traj.dict['bestStd'][:]

In [ ]:
traj = PTL.listTrajectories[1]
traj.dict['iField'][:]

In [ ]:
traj = PTL.listTrajectories[1]
np.array(traj.dict['idxRamp']) == 1

In [ ]:
PTL.nLoop*6*3*2

In [ ]:
plt.close('all')

In [ ]:
# fp = 'D://MagneticPincherData//Raw//21.04.27//21-04-27_M1_P1_C4_R40_disc20um_wFluo.tif'
# fp = "C://Users//JosephVermeil//Desktop//Fichier 3BiocheVsPhy@2x.png"
# fp = "C://Users//JosephVermeil//Desktop//21-04-27_M1_P1_C4_R40_disc20um_wFluo.tif"
# I = io.imread(fp)
# I

In [ ]:
a = ['a']
b = 'a'
len(a[0][0])
len(b[0][0])
isinstance(b, str)